**Are high special service requests flights also high-delay after controlling for load?**


In [16]:
import duckdb
con = duckdb.connect('../sql_databases/skyhack.duckdb')
con.execute("SHOW TABLES").df()

,name
0,PNRFlight
1,airports_cleaned
2,bagsData_cleaned
3,flights_cleaned
4,pnr_remarks_cleaned


In [17]:
sql_query_q5 = '''

-- 1) map record_locator -> flight/date using pnrFlight (pnrFlight contains record_locator -> flight_number + scheduled_departure_date_local)
WITH pnr_ref AS (
  SELECT
    UPPER(TRIM(record_locator)) AS record_locator,
    CAST(flight_number AS VARCHAR) AS flight_number,
    CAST(scheduled_departure_date_local AS DATE) AS dep_date
  FROM pnrFlight
  WHERE record_locator IS NOT NULL
),

-- 2) normalize pnrRemark entries and attach mapped dep_date when available
ssr_norm AS (
  SELECT
    UPPER(TRIM(r.record_locator)) AS record_locator,
    CAST(r.flight_number AS VARCHAR) AS flight_number,
    COALESCE(pf.dep_date, CAST(r.pnr_creation_date AS DATE)) AS dep_date_mapped,
    UPPER(TRIM(r.special_service_request)) AS ssr_text
  FROM pnr_remarks_cleaned r
  LEFT JOIN pnr_ref pf
    ON UPPER(TRIM(r.record_locator)) = pf.record_locator
    AND CAST(r.flight_number AS VARCHAR) = pf.flight_number
  WHERE r.special_service_request IS NOT NULL
),

-- 3) aggregate SSR counts per (flight_number, dep_date_mapped)
ssr_counts AS (
  SELECT
    flight_number,
    dep_date_mapped AS dep_date,
    COUNT(*) AS ssr_total,
    COUNT(DISTINCT record_locator) AS pnr_with_ssr
  FROM ssr_norm
  GROUP BY 1,2
),

-- 4) aggregate PNR pax per flight/date
pnr_agg AS (
  SELECT
    UPPER(TRIM(company_id)) AS company_id,
    CAST(flight_number AS VARCHAR) AS flight_number,
    CAST(scheduled_departure_date_local AS DATE) AS dep_date,
    SUM(COALESCE(total_pax,0)) AS pax_total
  FROM pnrFlight
  GROUP BY 1,2,3
),

-- 5) normalize flights and compute departure delay + keep ground-time fields
flight_base AS (
  SELECT DISTINCT
    UPPER(TRIM(company_id)) AS company_id,
    CAST(flight_number AS VARCHAR) AS flight_number,
    CAST(scheduled_departure_date_local AS DATE) AS dep_date,
    UPPER(TRIM(scheduled_departure_station_code)) AS dep_iata,
    UPPER(TRIM(scheduled_arrival_station_code)) AS arr_iata,
    COALESCE(total_seats, 0) AS total_seats,
    CAST(scheduled_departure_datetime_local AS TIMESTAMP) AS sched_dep_dt,
    CAST(actual_departure_datetime_local AS TIMESTAMP)    AS actual_dep_dt,
    COALESCE(scheduled_ground_time_minutes, NULL) AS scheduled_ground_time_minutes,
    COALESCE(actual_ground_time_minutes, NULL)    AS actual_ground_time_minutes,
    COALESCE(minimum_turn_minutes, NULL)          AS minimum_turn_minutes,
    carrier,
    fleet_type
  FROM flights_cleaned
  WHERE scheduled_departure_datetime_local IS NOT NULL
),

-- 6) join flight_base <- ssr_counts <- pnr_agg and compute metrics
joined AS (
  SELECT
    f.company_id,
    f.flight_number,
    f.dep_date,
    f.dep_iata,
    f.arr_iata,
    f.carrier,
    f.fleet_type,
    f.total_seats,
    COALESCE(p.pax_total, 0) AS pax_total,
    CASE WHEN f.total_seats > 0 THEN CAST(COALESCE(p.pax_total,0) AS DOUBLE)/f.total_seats ELSE NULL END AS load_factor,
    -- departure delay (minutes)
    ROUND((epoch(f.actual_dep_dt) - epoch(f.sched_dep_dt)) / 60.0, 1) AS dep_delay_mins,
    -- ground time comparisons
    f.scheduled_ground_time_minutes,
    f.actual_ground_time_minutes,
    CASE
      WHEN f.scheduled_ground_time_minutes IS NOT NULL AND f.actual_ground_time_minutes IS NOT NULL
      THEN ROUND((f.actual_ground_time_minutes - f.scheduled_ground_time_minutes), 1)
      ELSE NULL
    END AS ground_time_diff_mins,
    CASE
      WHEN f.scheduled_ground_time_minutes IS NOT NULL AND f.scheduled_ground_time_minutes <> 0
      THEN ROUND((CAST(f.actual_ground_time_minutes AS DOUBLE) / f.scheduled_ground_time_minutes), 3)
      ELSE NULL
    END AS ground_time_ratio,
    f.minimum_turn_minutes,
    COALESCE(s.ssr_total, 0) AS ssr_total,
    COALESCE(s.pnr_with_ssr, 0) AS pnr_with_ssr
  FROM flight_base f
  LEFT JOIN ssr_counts s
    ON f.flight_number = s.flight_number
    AND f.dep_date = s.dep_date
  LEFT JOIN pnr_agg p
    ON f.company_id = p.company_id
    AND f.flight_number = p.flight_number
    AND f.dep_date = p.dep_date
),

-- 7) categorize load and ssr levels (tunable thresholds)
categorized AS (
  SELECT *,
    CASE
      WHEN load_factor IS NULL THEN 'UNKNOWN'
      WHEN load_factor < 0.6 THEN 'LOW (<60%)'
      WHEN load_factor BETWEEN 0.6 AND 0.85 THEN 'MEDIUM (60–85%)'
      WHEN load_factor > 0.85 THEN 'HIGH (>85%)'
    END AS load_bin,
    CASE
      WHEN ssr_total = 0 THEN 'NO_SSR'
      WHEN ssr_total BETWEEN 1 AND 3 THEN 'LOW_SSR'
      ELSE 'HIGH_SSR'
    END AS ssr_bin,
    CASE WHEN dep_delay_mins > 0 THEN 1 ELSE 0 END AS is_late,
    CASE WHEN actual_ground_time_minutes IS NOT NULL AND minimum_turn_minutes IS NOT NULL AND actual_ground_time_minutes < minimum_turn_minutes THEN 1 ELSE 0 END AS actual_below_min_turn_flag
  FROM joined
)

-- 8) final summary by load_bin x ssr_bin
SELECT
  load_bin,
  ssr_bin,
  COUNT(*) AS flights,
  ROUND(AVG(dep_delay_mins), 2) AS avg_dep_delay_mins,
  ROUND(100.0 * SUM(is_late) / NULLIF(COUNT(dep_delay_mins),0), 2) AS pct_late,
  ROUND(AVG(ground_time_diff_mins), 2) AS avg_ground_time_diff_mins,
  ROUND(AVG(ground_time_ratio), 3) AS avg_ground_time_ratio,
  ROUND(AVG(ssr_total), 2) AS avg_ssr_per_flight,
  ROUND(AVG(pnr_with_ssr), 2) AS avg_pnrs_with_ssr,
  ROUND(AVG(load_factor), 3) AS avg_load_factor,
  ROUND(100.0 * SUM(actual_below_min_turn_flag) / NULLIF(SUM(CASE WHEN minimum_turn_minutes IS NOT NULL AND actual_ground_time_minutes IS NOT NULL THEN 1 ELSE 0 END),0), 2) AS pct_actual_below_min_turn
FROM categorized
WHERE dep_delay_mins IS NOT NULL
GROUP BY 1,2
ORDER BY
  CASE load_bin WHEN 'LOW (<60%)' THEN 1 WHEN 'MEDIUM (60–85%)' THEN 2 WHEN 'HIGH (>85%)' THEN 3 ELSE 4 END,
  CASE ssr_bin WHEN 'NO_SSR' THEN 1 WHEN 'LOW_SSR' THEN 2 WHEN 'HIGH_SSR' THEN 3 ELSE 4 END
;

'''
con.execute(sql_query_q5).df()


,load_bin,ssr_bin,flights,avg_dep_delay_mins,pct_late,avg_ground_time_diff_mins,avg_ground_time_ratio,avg_ssr_per_flight,avg_pnrs_with_ssr,avg_load_factor,pct_actual_below_min_turn
0,LOW (<60%),NO_SSR,147,34.84,41.50,-4.20,1.194,0.00,0.00,0.381,10.20
1,LOW (<60%),LOW_SSR,181,72.19,58.56,6.61,1.749,1.57,1.52,0.454,16.02
2,LOW (<60%),HIGH_SSR,19,100.00,84.21,38.42,1.167,4.53,3.95,0.508,26.32
3,MEDIUM (60–85%),NO_SSR,237,34.92,46.84,14.65,1.158,0.00,0.00,0.755,6.75
4,MEDIUM (60–85%),LOW_SSR,522,39.87,51.53,6.57,1.387,1.70,1.64,0.744,10.15
5,MEDIUM (60–85%),HIGH_SSR,116,61.12,59.48,11.67,1.182,5.13,4.64,0.766,12.93
6,HIGH (>85%),NO_SSR,1281,14.00,43.09,0.13,1.219,0.00,0.00,1.062,4.06
7,HIGH (>85%),LOW_SSR,3922,16.12,48.27,1.95,1.213,1.83,1.78,1.084,5.02
8,HIGH (>85%),HIGH_SSR,1640,20.56,56.89,6.45,1.235,5.61,5.22,1.125,3.23


In [18]:
con.close()